In [1]:
import random
from sklearn import linear_model
from matplotlib import pyplot as plt
from collections import defaultdict
import gzip
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score

import json
import numpy
import dateutil.parser
import math
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import scipy.optimize
from sklearn import svm
import string
import random

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from implicit.als import AlternatingLeastSquares
import numpy as np
import os
import tarfile

/home/mig009/.local/lib/python3.9/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## MovieLens Data

In [2]:
# Update the file paths based on your extraction path
extract_path = "ml-1m/ml-1m"

# Ratings File
ratings_file = os.path.join(extract_path, "ratings.dat")
ratings_columns = ["UserID", "MovieID", "Rating", "Timestamp"]
ratings_df = pd.read_csv(ratings_file, sep="::", engine="python", header=None, names=ratings_columns, encoding='latin1')

# Users File
users_file = os.path.join(extract_path, "users.dat")
users_columns = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
users_df = pd.read_csv(users_file, sep="::", engine="python", header=None, names=users_columns, encoding='latin1')

# Movies File
movies_file = os.path.join(extract_path, "movies.dat")
movies_columns = ["MovieID", "Title", "Genres"]
movies_df = pd.read_csv(movies_file, sep="::", engine="python", header=None, names=movies_columns, encoding='latin1')

## Last.FM (LFM360K) Data

In [3]:
tar_path = "lastfm-dataset-360K.tar.gz"
extract_path_lastfm = "lastfm-dataset-360K/lastfm-dataset-360K"

with tarfile.open(tar_path, 'r:gz') as tar_ref:
    tar_ref.extractall(extract_path_lastfm)

In [4]:
# Users-Artists-Plays File
users_artists_plays_file = os.path.join(extract_path_lastfm, "usersha1-artmbid-artname-plays.tsv")
users_artists_plays_columns = ["user-mboxsha1", "musicbrainz-artist-id", "artist-name", "plays"]
users_artists_plays_df = pd.read_csv(users_artists_plays_file, sep="\t", header=None, names=users_artists_plays_columns)

# Users Profile File
users_profile_file = os.path.join(extract_path_lastfm, "usersha1-profile.tsv")
users_profile_columns = ["user-mboxsha1", "gender", "age", "country", "signup"]
users_profile_df = pd.read_csv(users_profile_file, sep="\t", header=None, names=users_profile_columns)